In [1]:
import json
import re
import sys
import os
import pandas as pd
import ast

from tqdm import tqdm
from collections import Counter
from pathlib import Path
#from pandas import json_normalize #Deprecated? Also not used

In [4]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
import nltk
from gensim.utils import simple_preprocess#, lemmatize deprecetated
from gensim.models import CoherenceModel

from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt

from gensim.models import KeyedVectors

from nltk.corpus import stopwords
import gensim.downloader as api
import string
from sklearn.semi_supervised import LabelSpreading, LabelPropagation

# **`Clean and Process Data `**

In [5]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

      Unzipping corpora/stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [6]:
stop_words = stopwords.words('english')
stop_words.extend(["http", "https", "rt", "@", ":", "t.co", "co", "amp", "&amp;", "...", "\n", "\r"])
stop_words.extend(string.punctuation)

In [7]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True)
        yield(sent)

In [ ]:
# Look into what shape this ./abeled.json (misspelled?) is in so it can be replicated
df = pd.read_json("./abeled.json", orient='records',lines=True)
data = df.text.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

In [ ]:
data_ready = process_words(data_words)

In [ ]:
df['processed_text']=data_ready
df.to_json('./PR_all_Labeled.json', orient='records', lines=True)

In [ ]:
cp -a ./PR_all_Labeled.json  gdrive/My\ Drive/Code/

In [ ]:
data = pd.read_json("./PR_all_Labeled.json", orient='records',lines=True)

# Generating Similarity Scores and Matrix
### **Mean and Cosine Similarity(each event with all other event-types)**


In [ ]:
import pickle
#import logging
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.pipeline import make_pipeline
from scipy import sparse
import numpy as np
import os.path
import re
from tqdm import tqdm
import warnings
import pandas as pd

In [ ]:
# remove irrelevant tweets
data = pd.read_json("./PR_all_Labeled.json", orient='records',lines=True)
data.loc[(data.eventid == 'parisAttacks2015'),'event_type']='shooting'
def label_ir_tweets(postCategories):

    if 'Irrelevant' in postCategories:
        return 1
    else:
        return 0
data['ir'] = [label_ir_tweets(x) for x in data['categories']]
data=data.query("ir == 0")
data=data.query("label ==1")

In [ ]:
def label_observation_tweets(postCategories):
    if 'FirstPartyObservation' in postCategories or 'ThirdPartyObservation' in postCategories :
        return 1
    else:
        return 0
data['obs'] = [label_observation_tweets(x) for x in data['categories']]
data=data.query("obs == 1")
# data=data.query("label ==1")

In [ ]:
data['l'] = data.apply(lambda row: len(row['processed_text']), axis=1)
data= data.query("l >1")
data.drop(columns=['l'], inplace=True)

In [ ]:
pip install -U sentence-transformers

In [ ]:
# generate sentnece embedding
class SBERT:

    def __init__(self, lang="en"):
        from sentence_transformers import SentenceTransformer
        self.name = "SBERT"
        if lang == "fr":
            self.model = SentenceTransformer(
                "/home/bmazoyer/Dev/pytorch_bert/output/sts_fr_long_multilingual_bert-2019-10-01_15-07-03")
        elif lang == "en":
            self.model = SentenceTransformer(
                # "bert-large-nli-stsb-mean-tokens"
                "roberta-large-nli-stsb-mean-tokens"
            )
# roberta-large-nli-stsb-mean-tokens
    def compute_vectors(self, data):
        data["text"] = data.text.str.slice(0, 500)
        vectors = np.array(self.model.encode(data.text.tolist()))
        return vectors

In [ ]:
sbert=SBERT()

In [ ]:
v=sbert.compute_vectors(data)
data['sbert_emb']=[item for item in v]

In [ ]:
from numpy import dot
from numpy.linalg import norm

In [ ]:
def generate_similarity_matrix (frame, grouping, group_types):
    #generate similarity scores dataframe
    group_ranks=pd.DataFrame()
    groupby_col=grouping
    group_ranks=frame.query("label==1").groupby(grouping) #Does the label==1 need to change?
    for heldout_event,g in group_types:

        training = frame[frame[groupby_col] != heldout_event]
        test = frame[frame[groupby_col] == heldout_event]

        ref=np.mean(test["sbert_emb"], axis=0)

        grpups=training.query("label==1").groupby(grouping)
        ranks={}
        ranks["reference-group"]=heldout_event
        for name, group in grpups:
           val=np.mean(group["sbert_emb"], axis=0)
           cos_sim = dot(ref, val)/(norm(ref)*norm(val))
           ranks[name]=cos_sim

        # event_ranks[heldout_event]=ranks
        group_ranks = group_ranks.append(ranks, ignore_index=True)
    group_ranks.set_index("reference-group",inplace=True)
    return group_ranks


In [ ]:
import seaborn as sns

In [ ]:
#is this even required?
events=[ 
'2014_Philippines_Typhoon_Hagupi',
 '2015_Cyclone_Pam',
 'albertaFloods2013',
 'albertaWildfires2019',
 'australiaBushfire2013',
 'cycloneKenneth2019',
 'fireYMM2016',
 'hurricaneFlorence2018',
 'keralaFloods2019',
 'manilaFloods2013',
 'philipinnesFloods2012',
 'queenslandFloods2013',
 'southAfricaFloods2019',
 'typhoonHagupit2014',
 'typhoonYolanda2013'
]
event_types=[ 'hurricane/typhoon/cyclone/tornado','flood','wildfire/bushfire']

In [ ]:
event_ranks = generate_similarity_matrix(data, 'event_type', event_types)
event_ranks.to_csv("./event_ranks.csv")
#cp -a ./event_ranks_roberta.csv gdrive/My\ Drive/Code/

In [ ]:
# Visualize similarity matrix for event type using heatmap
event_ranks.set_index("reference-group",inplace=True)
p1 = sns.heatmap(event_ranks,cmap="YlGnBu", annot=True)

In [ ]:
info_types=[''] #double check that this is string based

In [ ]:
info_ranks = generate_similarity_matrix(data, 'info_type', info_types) #check what this variable is called in the dataframe
info_ranks.to_csv("./info_ranks.csv")

In [ ]:
# Visualize similarity matrix for info type using heatmap
info_ranks.set_index("reference-group",inplace=True)
p2 = sns.heatmap(info_ranks,cmap="YlGnBu", annot=True)

In [ ]:
critical_types=[0, 1, 2, 3] #double check this

In [ ]:
critical_ranks = generate_similarity_matrix(data, 'criticality_type', critical_types) #check what this variable is called in the dataframe
critical_ranks.to_csv("./critical_ranks.csv")

In [ ]:
# Visualize similarity matrix for info type using heatmap
critical_ranks.set_index("reference-group",inplace=True)
p3 = sns.heatmap(critical_ranks,cmap="YlGnBu", annot=True)